# Fine-mapping with SuSiE RSS model

This notebook mainly responsible for a matrixs of RSS using both mvsusie and a loop of uni_susie

3. GWAS summary statistics input `z` and `R`. We assume `z` scores have been computed after removal of covariates `C`.

## Input

1. A recipe file documenting the path to

1.1. A list of analysis_unit_list generated from Meta_Analysis step for each gene.

    Each line of the index record the name of 1 RDS file, containing a list with bhat and sbhat object matrixs, with dimension = (number of snps*number of theme) 
    
1.2.The path to a residual corr file from meta_analysis step [FIXME: Specified resid_corr for each gene to be added in the future]

1.3. 3 col each record the path to a prior file from meta_analysis step.

2. LD Recipe: a three colnums table with Theme as each of the Theme_prefix and other columns =  ld_file_prefix,   ld_file_surfix

Only SNPS presented in both the LD matrixs and the sumstat rds will be analysis. 
In unisusie_rss, each of theme will use the corresponding ld file
In mvsusie_rss, the last row will be used.

The ld_prefix and ld_surfix are such that paste0(ld_file_prefix,gene_ID,ld_file_surfix) generate the path to each ld         matrix for each genes

## Output

For each analysis unit we output:

1. Analysis results in RDS format: A mvsusie Model
2. A vcf file with selected snps
        ES:PIP:CS

## examples



In [1]:
[global]
import glob
import pandas as pd
# Reciepe
parameter: recipe = path("./")
#file_inv = pd.read_csv(recipe, sep = "\t")
parameter: merged_analysis_unit = path#(file_inv["merged_analysis_unit"].values.tolist()[0])
#parameter: resid_cor = path(file_inv["resid_corr"].values.tolist()[0])
parameter: Theme_prefix = str #file_inv["Theme_prefix"].values.tolist()[0]
## LD Recipe: a three colnums table with n_themes rows, as only 1 LD is required for each gene and each theme for uni_susie_RSS, columns =  ld_file_prefix, ld_file_surfix, Theme
parameter: LD_Recipe = path
## The snp with pip > criterior will be reported
parameter: pip_criterior = 0.1
#parameter: prior = path(file_inv["prior"].values.tolist()[0])
## data file suffix
parameter: data_suffix = ""
#
## An identifier for your run of analysis
parameter: name = Theme_prefix
#
regions = [x.replace("\"","").strip().split() for x in open(merged_analysis_unit).readlines() if x.strip() and not x.strip().startswith('#')]
genes = regions
## Path to work directory where output locates
parameter: wd = path("./output")
## Containers that contains the necessary packages
parameter: container = "/mnt/mfs/statgen/containers/twas_latest.sif"
## Only 1 LD for each gene is required for each analysis


In [ ]:
[Fine_mapping_1,summary_stats_preprocessing]
parameter: ld_type = 'original'
parameter: bhat = "bhat"
parameter: sbhat = "sbhat"
input: genes, group_by = 1
output: processed = f'{wd:a}/preprocessed/{_input:b}'
task: trunk_workers = 1, trunk_size = 1, walltime = '2h', mem = '55G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container
    library("dplyr")
    library("readr")
    dat = readRDS(${_input:r})
    if(is.null(dat$Z)){
    dat$Z = dat$${bhat}/dat$${sbhat}
      }
    gene_id = read.table(text = "${_input:bnn}", sep = "_")
    gene_id = gene_id[,ncol(gene_id)][[1]]
    ld.table = read_delim("${LD_Recipe}" , "\t")%>%mutate(ld.path = paste0(ld_file_prefix, gene_id ,ld_file_surfix )) 
    rownames(ld.table) = ld.table$Theme
    ld.file = ld.table[nrow(ld.table),]
    dat$ld.file = ld.file
    dat$ld.table = ld.table
    saveRDS(dat, ${_output:r})

In [ ]:
[Fine_mapping_2,MvSuSiE_summary_stats_analysis_1]
parameter: max_L = 10
parameter: max_iter = 10
parameter: ld_type = 'original'
parameter: bhat = "bhat"
parameter: sbhat = "sbhat"
parameter: resid_cor = path#(file_inv["resid_corr"].values.tolist()[0])
parameter: prior = path#(file_inv["prior"].values.tolist()[0])
input: output_from("summary_stats_preprocessing")
output: f'{wd:a}/{_input:bnn}.LD{ld_type}{resid_cor:bnx}.mvsusierss.model.rds'
task: trunk_workers = 1, trunk_size = 1, walltime = '2h', mem = '55G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container
    get_prior_indices <- function(Z, U) {
      # make sure the prior col/rows match the colnames of the Y matrix
      z_names = colnames(Z)
      u_names = colnames(U)
      if (is.null(z_names) || is.null(u_names)) {
          return(NULL)
      } else if (identical(z_names, u_names)) {
          return(NULL)
      } else {
          return(match(z_names, u_names))
      }
    }

    library(mvsusieR)
    library("dplyr")
    library("tibble")
    library("purrr")
    library("readr")
    library("tidyr")
    dat = readRDS(${_input:r})
    gene_id = "${_input:bnn}"
    resid_cor = ${resid_cor:r}
    V = readRDS(resid_cor)
    prior = readRDS(${prior:r})
    print(paste("Number of components in the mixture prior:", length(prior$U)))
    prior = mvsusieR::create_mash_prior(mixture_prior=list(weights=prior$w, matrices=prior$U), 
                                        include_indices = get_prior_indices(dat$Z, prior$U[[1]]), 
                                        max_mixture_len=-1)
    if("${ld_type}" == 'original'){
        R = readRDS(dat$ld.file$ld.path)
    }else if("${ld_type}" == 'remove_cov'){
        R = dat$LD
    }
  
    ## Remove the NA SNPs in R, assuming all NA = 0
    R[is.na(R)] = 0
    
    ## Remove duplicated in Z
    dat$Z = dat$Z[!duplicated(dat$snps),]
    dat$snps = dat$snps[!duplicated(dat$snps)]
    
    # Retaining only the overlapping snps

    R = R[which(rownames(R)%in%dat$snps),which(colnames(R)%in%dat$snps)]
    dat$Z = dat$Z[which(dat$snps%in%rownames(R)),]
    dat$snps = dat$snp[which(dat$snps%in%rownames(R))]


    ## Initiate the computation  
    st = proc.time()
    mv_res = mvsusieR::mvsusie_rss(dat$Z, R, L=${max_L}, 
                                   prior_variance=prior, residual_variance=V, 
                                   precompute_covariances=T, compute_objective=T, 
                                   estimate_prior_variance=T, estimate_prior_method='EM',
                                   max_iter = ${max_iter}, n_thread=1)
    mv_res$time = proc.time() - st
    #if(mv_res$convergence$converged == FALSE){
    #    stop('Fail to converge.')
    #}
    mv_res$cs_corr = susieR:::get_cs_correlation(mv_res, Xcorr=R)
  
    # Get list of cs snps
    mv_output = mv_res
  
    saveRDS(mv_res, ${_output[0]:r})

In [ ]:
[Fine_mapping_3,MvSuSiE_summary_stats_analysis_2]
input:  group_by = 1
output:    f'{_input:nn}.result.vcf.bgz'
task: trunk_workers = 1, trunk_size = 1, walltime = '2h', mem = '55G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output[0]:n}.stdout", stderr = f"{_output[0]:n}.stderr"
    library("dplyr")
    library("tibble")
    library("purrr")
    library("readr")
    library("tidyr")
    mv_res = readRDS(${_input:r})
    ## Define create_vcf function
           create_vcf = function (chrom, pos, nea, ea, snp = NULL, ea_af = NULL, effect = NULL, 
        se = NULL, pval = NULL, name = NULL,cs = NULL, pip = NULL) 
    {
        stopifnot(length(chrom) == length(pos))
        if (is.null(snp)) {
            snp <- paste0(chrom, ":", pos)
        }
        snp <- paste0(chrom, ":", pos)
        nsnp <- length(chrom)
        gen <- list()
        ## Setupt data content for each sample column
        if (!is.null(ea_af)) 
            gen[["AF"]] <- matrix(ea_af, nsnp)
        if (!is.null(effect)) 
            gen[["ES"]] <- matrix(effect, nsnp)
        if (!is.null(se)) 
            gen[["SE"]] <- matrix(se, nsnp)
        if (!is.null(pval)) 
            gen[["LP"]] <- matrix(-log10(pval), nsnp)
        if (!is.null(cs)) 
            gen[["CS"]] <- matrix(cs, nsnp)
        if (!is.null(pip)) 
            gen[["PIP"]] <- matrix(pip, nsnp)
        gen <- S4Vectors::SimpleList(gen)
        
      ## Setup snps info for the fix columns
        gr <- GenomicRanges::GRanges(chrom, IRanges::IRanges(start = pos, 
            end = pos + pmax(nchar(nea), nchar(ea)) - 1, names = snp))
         coldata <- S4Vectors::DataFrame(Studies = name, row.names = name)
    ## Setup header informations
        hdr <- VariantAnnotation::VCFHeader(header = IRanges::DataFrameList(fileformat = S4Vectors::DataFrame(Value = "VCFv4.2", 
            row.names = "fileformat")), sample = name)
        VariantAnnotation::geno(hdr) <- S4Vectors::DataFrame(Number = c("A", 
            "A", "A", "A", "A", "A"), Type = c("Float", "Float", 
            "Float", "Float", "Float", "Float"), Description = c("Effect size estimate relative to the alternative allele", 
            "Standard error of effect size estimate", "-log10 p-value for effect estimate",  
            "Alternate allele frequency in the association study",
            "The CS this variate are captured, 0 indicates not in any cs", "The posterior inclusion probability to a CS"), 
            row.names = c("ES", "SE", "LP", "AF", "CS", "PIP"))
      ## Save only the meta information in the sample columns 
        VariantAnnotation::geno(hdr) <- subset(VariantAnnotation::geno(hdr), 
            rownames(VariantAnnotation::geno(hdr)) %in% names(gen))
      ## Save VCF 
        vcf <- VariantAnnotation::VCF(rowRanges = gr, colData = coldata, 
            exptData = list(header = hdr), geno = gen)
        VariantAnnotation::alt(vcf) <- Biostrings::DNAStringSetList(as.list(ea))
        VariantAnnotation::ref(vcf) <- Biostrings::DNAStringSet(nea)
      ## Add fixed values
        VariantAnnotation::fixed(vcf)$FILTER <- "PASS"
          return(sort(vcf))
        }
    
    # Get list of cs snps
    mv_output_snps = tibble( snps =  mv_res$variable_names[which(mv_res$pip > 0)], snps_index = which((mv_res$pip > 0))  )
    mv_output_snps = mv_output_snps%>%mutate( cs = map(snps_index,~which(mv_res$sets$cs %in% .x))%>%as.numeric%>%replace_na(0),
                             pip = map_dbl(snps_index,~(mv_res$pip[.x])),
                     chr = map_chr(snps,~read.table(text = .x,sep = ":",as.is = T)$V1),
                     pos_alt_ref = map_chr(snps,~read.table(text = .x,sep = ":",as.is = TRUE)$V2),
                     pos = map_dbl(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE)$V1),
                     alt = map_chr(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE, colClass = "character")$V2),
                     ref = map_chr(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE, colClass = "character")$V3))
    
    effect_mtr = mv_res$coef[mv_output_snps$snps_index+1,]
    colnames(effect_mtr) = mv_res$condition_names
    rownames(effect_mtr) = mv_output_snps$snps
    cs_mtr = effect_mtr
    for(i in 1:nrow(cs_mtr)) cs_mtr[i,] =  mv_output_snps$cs[[i]]  
    pip_mtr = effect_mtr
    for(i in 1:nrow(pip_mtr)) pip_mtr[i,] =  mv_output_snps$pip[[i]]  

    output_vcf = create_vcf(
           chrom = mv_output_snps$chr,
            pos = mv_output_snps$pos,
            ea = mv_output_snps$alt,
            nea = mv_output_snps$ref,
            effect = effect_mtr ,
            pip = pip_mtr,
            cs = cs_mtr,
            name = colnames(effect_mtr)
              )
  
    saveRDS(mv_output_snps, "${_output[0]:nn}.rds")
    VariantAnnotation::writeVcf(output_vcf,${_output[0]:nr},index = TRUE)

In [ ]:
[Fine_mapping_4,MvSuSiE_summary_stats_analysis_3]
input: group_by = "all"
output: f'{wd}/{Theme_prefix}.mvsusie_rss.output_list.txt'
python: expand= "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
    import pandas as pd
    pd.DataFrame({"output_vcf" : [$[_input:ar,]]}).to_csv("$[_output]",index = False ,header = False, sep = "t")

## Univariate SuSiE RSS

In [1]:
[Fine_mapping_5,UniSuSiE_summary_stats_analysis_1]
parameter: max_L = 10
parameter: ld_type = 'original'
parameter: bhat = "bhat"
parameter: sbhat = "sbhat"
input: output_from("summary_stats_preprocessing")["processed"]
output: uni_rds = f'{wd:a}/{_input:bnn}.LD{ld_type}.unisusierss.model.rds',
        uni_vcf = f'{wd:a}/{_input:bnn}.LD{ld_type}.unisusierss.vcf.bgz'
task: trunk_workers = 1, trunk_size = 1, walltime = '2h', mem = '55G', cores = 1, tags = f'{step_name}_{_output[0]:bn}'
R: expand = '${ }', stdout = f"{_output[0]:n}.stdout", stderr = f"{_output[0]:n}.stderr"
    library("susieR")
    library("dplyr")
    library("tibble")
    library("purrr")
    library("readr")
    library("tidyr")
  
     ## Define create_vcf function
           create_vcf = function (chrom, pos, nea, ea, snp = NULL, ea_af = NULL, effect = NULL, 
        se = NULL, pval = NULL, name = NULL,cs = NULL, pip = NULL) 
    {
        stopifnot(length(chrom) == length(pos))
        if (is.null(snp)) {
            snp <- paste0(chrom, ":", pos)
        }
        snp <- paste0(chrom, ":", pos)
        nsnp <- length(chrom)
        gen <- list()
        ## Setupt data content for each sample column
        if (!is.null(ea_af)) 
            gen[["AF"]] <- matrix(ea_af, nsnp)
        if (!is.null(effect)) 
            gen[["ES"]] <- matrix(effect, nsnp)
        if (!is.null(se)) 
            gen[["SE"]] <- matrix(se, nsnp)
        if (!is.null(pval)) 
            gen[["LP"]] <- matrix(-log10(pval), nsnp)
        if (!is.null(cs)) 
            gen[["CS"]] <- matrix(cs, nsnp)
        if (!is.null(pip)) 
            gen[["PIP"]] <- matrix(pip, nsnp)
        gen <- S4Vectors::SimpleList(gen)
        
      ## Setup snps info for the fix columns
        gr <- GenomicRanges::GRanges(chrom, IRanges::IRanges(start = pos, 
            end = pos + pmax(nchar(nea), nchar(ea)) - 1, names = snp))
         coldata <- S4Vectors::DataFrame(Studies = name, row.names = name)
    ## Setup header informations
        hdr <- VariantAnnotation::VCFHeader(header = IRanges::DataFrameList(fileformat = S4Vectors::DataFrame(Value = "VCFv4.2", 
            row.names = "fileformat")), sample = name)
        VariantAnnotation::geno(hdr) <- S4Vectors::DataFrame(Number = c("A", 
            "A", "A", "A", "A", "A"), Type = c("Float", "Float", 
            "Float", "Float", "Float", "Float"), Description = c("Effect size estimate relative to the alternative allele", 
            "Standard error of effect size estimate", "-log10 p-value for effect estimate",  
            "Alternate allele frequency in the association study",
            "The CS this variate are captured, 0 indicates not in any cs", "The posterior inclusion probability to a CS"), 
            row.names = c("ES", "SE", "LP", "AF", "CS", "PIP"))
      ## Save only the meta information in the sample columns 
        VariantAnnotation::geno(hdr) <- subset(VariantAnnotation::geno(hdr), 
            rownames(VariantAnnotation::geno(hdr)) %in% names(gen))
      ## Save VCF 
        vcf <- VariantAnnotation::VCF(rowRanges = gr, colData = coldata, 
            exptData = list(header = hdr), geno = gen)
        VariantAnnotation::alt(vcf) <- Biostrings::DNAStringSetList(as.list(ea))
        VariantAnnotation::ref(vcf) <- Biostrings::DNAStringSet(nea)
      ## Add fixed values
        VariantAnnotation::fixed(vcf)$FILTER <- "PASS"
          return(sort(vcf))
        }
  
    dat = readRDS(${_input:r})
    gene_id = "${_input:bnn}"

    ## Initiate the computation  
    st = proc.time()
    susie_list = list()
    for (i in 1:ncol(dat$Z)) {
          ## Retaining only the overlapping snps
    ld = readRDS(dat$ld.table[colnames(dat$Z)[i],]$ld.path)
    int_snps = intersect(rownames(ld),dat$snps)
    ld = ld[which(rownames(ld)%in%int_snps),which(colnames(ld)%in%int_snps)]
    ld[is.na(ld)] = 0
    Z = dat$Z[which(dat$snps%in%int_snps),i]
    snps = dat$snps[which(dat$snps%in%int_snps)]
      susie_list[[i]] = susie_rss(Z, ld)
      susie_list[[i]]$conditions_name = colnames(dat$Z)[[i]]
      susie_list[[i]]$variable_name = snps
      }

  
      susie_tb_ls = list()
    for (i in 1:length(susie_list)){
        susie_tb = tibble( snps =  susie_list[[i]]$variable_name[which( susie_list[[i]]$pip >= 0)], snps_index = which(( susie_list[[i]]$pip >= 0))  )
        susie_tb_ls[[i]]= susie_tb%>%mutate( cs = map(snps_index,~which( susie_list[[i]]$sets$cs %in% .x))%>%as.numeric%>%replace_na(0),
                                 pip = map_dbl(snps_index,~( susie_list[[i]]$pip[.x])),
                                 coef = map_dbl(snps_index,~(coef.susie( susie_list[[i]])[.x+1])))
        }
    for(i in 2:length(susie_tb_ls)){susie_tb_ls[[i]] = full_join(susie_tb_ls[[i-1]],susie_tb_ls[[i]], by = "snps") }
    m = c("cs","pip","coef")    
    output = list()
    for(i in m){
    output[[i]] = susie_tb_ls[[length(susie_tb_ls)]]%>%select(contains(i))%>%as.matrix
    }
    snps_tb = susie_tb_ls[[length(susie_tb_ls)]]%>%mutate(
                         chr = map_chr(snps,~read.table(text = .x,sep = ":",as.is = T)$V1),
                         pos_alt_ref = map_chr(snps,~read.table(text = .x,sep = ":",as.is = TRUE)$V2),
                         pos = map_dbl(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE)$V1),
                         alt = map_chr(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE, colClass = "character")$V2),
                         ref = map_chr(pos_alt_ref,~read.table(text = .x,sep = "_",as.is = TRUE, colClass = "character")$V3))
  
    output_vcf = create_vcf(
               chrom = snps_tb$chr,
                pos = snps_tb$pos,
                ea = snps_tb$alt,
                nea = snps_tb$ref,
                effect = output$coef ,
                pip = output$pip,
                cs = output$cs,
                name = colnames(dat$Z)
                  )
  
      
    saveRDS(susie_list, ${_output[0]:r})
    VariantAnnotation::writeVcf(output_vcf,${_output[1]:nr},index = TRUE)

In [ ]:
[Fine_mapping_6,UniSuSiE_summary_stats_analysis_2]
input: output_from("UniSuSiE_summary_stats_analysis_1")["uni_vcf"], group_by = "all"
output: f'{wd}/{Theme_prefix}.unisusie_rss.output_list.txt'
python: expand= "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
    import pandas as pd
    pd.DataFrame({"output_vcf" : [$[_input:ar,]]}).to_csv("$[_output]",index = False ,header = False, sep = "t")